# 前処理100本ノック（Pandas）

In [1]:
# ライブラリのimport
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split, TimeSeriesSplit


# dtypeの設定
dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str,
    'application_date': str,
    'birth_day': str
}

# DataFrameの作成
df_category = pd.read_csv("../data/category.csv", dtype=dtypes)
df_customer = pd.read_csv("../data/customer.csv", dtype=dtypes)
df_geocode  = pd.read_csv("../data/geocode.csv", dtype=dtypes)
df_product  = pd.read_csv("../data/product.csv", dtype=dtypes)
df_receipt  = pd.read_csv("../data/receipt.csv", dtype=dtypes)
df_store    = pd.read_csv("../data/store.csv", dtype=dtypes)

# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


In [3]:
# スライスでも可能
df_receipt[:10]

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [4]:
df_002 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]

df_002.head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [5]:
df_002 = df_receipt.loc[:, ["sales_ymd", "customer_id", "product_cd", "amount"]]

df_002.head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、5件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [6]:
# renameは辞書形式で行う
# rename(columns={変化前のcolumn名: 変化後のcolumn名})
df_003 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]].rename(columns={"sales_ymd":"sales_date"})

df_003.head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


## 抽出

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [7]:
# queryを使う方法

df_004 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]
df_004 = df_004.query("customer_id == 'CS018205000001'")

df_004.head(10)

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [8]:
# queryを使わない方法
df_004 = df_receipt[df_receipt["customer_id"]=="CS018205000001"]
df_004 = df_004[["sales_ymd", "customer_id", "product_cd", "amount"]]


df_004.head(10)

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [9]:
# queryを使う方法

df_005 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]
df_005 = df_005.query('customer_id == "CS018205000001" & amount >= 1000')

df_005.head()

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [10]:
# 別解
# queryを使わない方法
# &や|といった複数条件を扱う場合は()によって条件を分割する

df_005 = df_receipt[
    (df_receipt["customer_id"] == "CS018205000001")
    & (df_receipt["amount"] >= 1000)
]
df_005 = df_005[["sales_ymd", "customer_id", "product_cd", "amount"]]


df_005.head()

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [11]:
df_006 = df_receipt[["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]
df_006 = df_006.query('customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 1000)')

df_006.head()

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


In [12]:
# 別解
df_006 = df_receipt[
    (df_receipt["customer_id"] == "CS018205000001")
    & ((df_receipt["amount"] >= 1000) | (df_receipt["quantity"] >= 1000))
]
df_006 = df_006[["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]

df_006.head()

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [13]:
df_007 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]
df_007 = df_007.query('customer_id == "CS018205000001" & (1000 <= amount <= 2000)')

df_007.head()

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


In [14]:
# 別解
df_007 = df_receipt[
    (df_receipt["customer_id"] == "CS018205000001")
    & ((1000 <= df_receipt["amount"]) & (df_receipt["amount"] <= 2000))
]
df_007 = df_007[["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]

df_007.head()

,sales_ymd,customer_id,product_cd,quantity,amount
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [15]:
df_008 = df_receipt[["sales_ymd", "customer_id", "product_cd", "amount"]]
df_008 = df_008.query('customer_id == "CS018205000001" & product_cd != "P071401019"')

df_008

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [16]:
# 別解
df_008 = df_receipt[
    (df_receipt["customer_id"] == "CS018205000001")
    & (df_receipt["product_cd"] != "P071401019")
]
df_008 = df_008[["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]

df_008

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
27673,20170614,CS018205000001,P060702015,1,108
27840,20190216,CS018205000001,P071005024,1,102
28757,20180414,CS018205000001,P071101002,1,278
39256,20190226,CS018205000001,P070902035,1,168
58121,20190924,CS018205000001,P060805001,1,495
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [17]:
df_009 = df_store.query('prefecture_cd != "13" & floor_area <= 900')

df_009

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [18]:
# 別解
df_009 = df_store[
    (df_store["prefecture_cd"] != "13")
    & (df_store["floor_area"] <= 900)
]

df_009

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、5件表示せよ。

文字列の条件
| メソッド            | 処理                    |
| --------------- | --------------------- |
| str.starts_with | 指定文字列が先頭にあるか          |
| str.ends_with   | 指定文字列が終端にあるか          |
| str.contains    | 指定文字列を含むかどうか(正規表現に対応) |

In [19]:
df_010 = df_store.query('store_cd.str.startswith("S14")', engine="python")  # 文字列メソッドはnumexprをインストールしている場合はengine=pythonとする必要がある


df_010.head(5)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0


In [20]:
# 別解
df_010 = df_store[df_store["store_cd"].str.startswith("S14")]

df_010.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0


---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、5件表示せよ。

In [21]:
df_011 = df_customer.query("customer_id.str.endswith('1')",engine="python")


df_011.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4


In [22]:
# 別解
df_010 = df_customer[df_customer["customer_id"].str.endswith("1")]

df_010.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4


---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [23]:
df_012 = (
    df_store
    .query("address.str.contains('横浜市')")
)

df_012

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、5件表示せよ。

先頭の文字は`^`で表す(https://qiita.com/luohao0404/items/7135b2b96f9b0b196bf3 参照)

In [24]:
df_013 = df_customer.query("status_cd.str.contains(r'^[A-F]')", engine="python")

df_013.head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F


In [25]:
df_013 = df_customer[
    df_customer["status_cd"].str.contains("^[A-F]")  # pythonの正規表現
]

df_013.head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F


---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、5件表示せよ。

最後尾の文字は`$`で表す

In [26]:
df_014 = df_customer.query("status_cd.str.contains('[1-9]$')", engine="python")


df_014.head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1


In [27]:
df_014 = df_customer[df_customer["status_cd"].str.contains("[1-9]$")]

df_014.head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1


---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、5件表示せよ。

In [28]:
df_015 = df_customer.query("status_cd.str.contains('^[A-F].*[1-9]$')")


df_015.head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3


In [29]:
df_015 = df_customer[df_customer["status_cd"].str.contains("^[A-F].*[1-9]$")]

df_015.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3


---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [30]:
df_016 = df_store.query("tel_no.str.contains('[0-9]{3}-[0-9]{3}-[0-9]{4}')")


df_016

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


In [31]:
# 別解
df_016 = df_store[df_store["tel_no"].str.contains("\d{3}-\d{3}-\d{4}")]

df_016

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


## ソート

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を5件表示せよ。

In [32]:
df_017 = df_customer.sort_values(by="birth_day", ascending=True)


df_017.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0


---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を5件表示せよ。

In [33]:
# 引数にascenging=Falseを指定すると降順になる
df_018 = df_customer.sort_values("birth_day", ascending=False)


df_018.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

ランク付けにはエクスプレッションに対して`.rank()`メソッドを用いる。引数に与えるパラメータとして以下がある
- `average`:同じ値のものはランクの平均値になる(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3.5, 3.5, 5.5, 5.5, 7)
- `min`:同じ値のものはランクは最小値で表す(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3, 3, 5, 5, 7)
- `max`:同じ値のものはランクは最大値で表す(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 4, 4, 6, 6, 7)
- `dense`:`min`と似ているが、割り当てられるランクはその前のランクの次の値になる(例:1, 2, 3, 3, 4, 4, 5) -> rank(1, 2, 3, 3, 4, 4, 5)
- `first`:`min`と似ているが、同値は値の出現順で重複なしでランクを与えられる

In [34]:
df_tmp = pd.concat(
    [
        df_receipt[["customer_id", "amount"]],
        df_receipt["amount"].rank(method="min", ascending=False)
    ],
    axis=1
)

df_tmp.columns = ["customer_id", "amount", "ranking"]
df_019 = df_tmp.sort_values("ranking")

df_019.head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [35]:
df_tmp = pd.concat(
    [
        df_receipt[["customer_id", "amount"]],
        df_receipt["amount"].rank(method="first", ascending=False)
    ],
    axis=1
)

df_tmp.columns = ["customer_id", "amount", "ranking"]
df_019 = df_tmp.sort_values("ranking")

df_019.head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


## 集計

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [36]:
len(df_receipt)

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [37]:
len(df_receipt["customer_id"].unique())

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [38]:
df_023 = df_receipt.groupby("store_cd").aggregate({"amount":"sum", "quantity":"sum"}).reset_index()  # groupbyをするとindexが消えるため、reset_index()で付け直す必要がある

df_023.head(10)

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、5件表示せよ。

In [39]:
df_024 = df_receipt.groupby("customer_id").aggregate({"sales_ymd":"max"}).reset_index()

df_024.head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [40]:
df_025 = df_receipt.groupby("customer_id").aggregate({"sales_ymd":"min"}).reset_index()

df_025.head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [41]:
df_026 = df_receipt.groupby("customer_id").aggregate({"sales_ymd":["min", "max"]}).reset_index()
df_026.columns=["customer_id", "sales_ymd_min", "sales_ymd_max"]
df_026 = df_026[df_026["sales_ymd_max"] != df_026["sales_ymd_min"]]

df_026.head(10)

,customer_id,sales_ymd_min,sales_ymd_max
1,CS001114000005,20180503,20190731
2,CS001115000010,20171228,20190405
3,CS001205000004,20170914,20190625
4,CS001205000006,20180207,20190224
13,CS001214000009,20170306,20190902
14,CS001214000017,20180828,20191006
16,CS001214000048,20171109,20190929
17,CS001214000052,20180208,20190617
20,CS001215000005,20170206,20181021
21,CS001215000040,20170214,20171022


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [42]:
df_027 = (
    df_receipt
    .groupby("store_cd").aggregate({"amount":"mean"})  # 売上金額の平均
    .reset_index()                               # indexを付与
    .sort_values("amount", ascending=False)      # 降順でソート
)

df_027.head(5)  # top5を表示

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [43]:
df_028 = (
    df_receipt
    .groupby("store_cd").aggregate({"amount":"median"})  # 売上金額の中央値
    .reset_index()
    .sort_values("amount", ascending=False)
)

df_028.head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [44]:
df_029 = (
    df_receipt
    .groupby("store_cd")["product_cd"].apply(lambda x:x.mode())  # mode(最頻値)の計算
    .reset_index()
)

df_029.head(10)

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [45]:
df_030 = (
    df_receipt
    .groupby("store_cd")["amount"].var(ddof=0)  # amountの分散を計算
    .reset_index()
    .sort_values("amount", ascending=False)
)

df_030.head(5)


,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [46]:
df_031 = (
    df_receipt
    .groupby("store_cd")["amount"].std(ddof=0)  # amountの標準偏差を計算
    .reset_index()
    .sort_values("amount", ascending=False)
)
df_031.head(5)

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [47]:
df_032 = df_receipt["amount"].quantile(q=[0.25, 0.5, 0.75, 1.0])

df_032

0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

In [48]:
# numpy利用
np.percentile(df_receipt["amount"], q=[25, 50, 75, 100])

array([  102.,   170.,   288., 10925.])

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [49]:
df_033 = (
    df_receipt
    .groupby("store_cd").aggregate({"amount":"mean"})
    .query("amount >= 330")
)
df_033

,amount
store_cd,
S12013,330.194130
S13001,348.470386
S13003,350.915519
S13004,330.943949
S13015,351.111960
S13019,330.208616
S13020,337.879932
S13052,402.867470
S14010,348.791262


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [50]:
df_034 = (
    df_receipt
    .query("not(customer_id.str.startswith('Z'))")
    .groupby("customer_id").aggregate({"amount":"sum"})["amount"]
    .mean()
)

df_034

2547.742234529256

In [51]:
# query+aggを使わない場合
df_034 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id")["amount"].sum()
    .mean()
)

df_034

2547.742234529256

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [52]:
df_035 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id")["amount"].sum()
    .reset_index()
)

df_035 = df_035[df_035["amount"] >= df_035["amount"].mean()]

df_035.head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


## DataFrameの結合

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

- 内部結合(inner):結合させる両方のDataFrameに存在する結合キー(on)で、一致する項目のみを抽出する結合方法
- 左外部結合(left):結合させる両方のDataFrameに存在する結合キー(on)で、基準となるDataFrameを左とする。基準DataFrameはすべての行が出力され、結合されたDataFrameは結合キーの値が基準の結合キーと一致した行のみ出力される
- 完全外部結合(outer):結合させる両方のDataFrameに存在する結合キー(on)で、基準となるDataFrameを左とする。基準のDataFrameおよび結合されたDataFrameのすべての行が出力される

In [53]:
df_036 = pd.merge(
    left=df_receipt,
    right=df_store[["store_cd", "store_name"]],
    how="inner",   # 内部結合
    on="store_cd"  # store_cdを基準に結合
)

df_036.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [54]:
df_037 = pd.merge(
    left=df_product,
    right=df_category[["category_small_cd", "category_small_name"]],
    how="inner",
    on="category_small_cd"
)


df_037.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [55]:
df_038 = pd.merge(
    left=df_customer[
        (df_customer["gender_cd"] == "1")
        & (~df_customer["customer_id"].str.startswith("Z"))
    ]["customer_id"],
    right=df_receipt.groupby("customer_id")["amount"].sum().reset_index(),
    how="left",  # leftを基準に
    on="customer_id"
).fillna(0)

df_038.head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


In [56]:
# 別解
df_tmp = df_customer.query("gender_cd == '1' and not customer_id.str.startswith('Z')",
                            engine="python")

df_038 = pd.merge(df_tmp["customer_id"],
                  df_receipt.groupby("customer_id").amount.sum().reset_index(),
                  how="left", on="customer_id").fillna(0)

df_038.head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [57]:
df_tmp = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]

# 売り上げ日数の多い顧客上位20位
df_cnt = (
    df_tmp[~df_tmp.duplicated(subset=["customer_id", "sales_ymd"])]
    .groupby("customer_id")["sales_ymd"].count()
    .reset_index()
    .sort_values("sales_ymd", ascending=False)
    .head(20)
)

# 売り上げ金額合計の多い顧客上位20件
df_sum = (
    df_tmp
    .groupby("customer_id")["amount"].sum()
    .reset_index()
    .sort_values("amount", ascending=False)
    .head(20)
)

df_039 = pd.merge(df_cnt, df_sum, how="outer", on="customer_id")

df_039

,customer_id,sales_ymd,amount
0,CS040214000008,23.0,NaN
1,CS015415000185,22.0,20153.0
2,CS010214000010,22.0,18585.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0
5,CS017415000097,20.0,23086.0
6,CS016415000141,20.0,18372.0
7,CS031414000051,19.0,19202.0
8,CS014214000023,19.0,NaN
9,CS022515000226,19.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [58]:
# df_storeとdf_productには共通のkeyがないので作成する
df_040 = pd.merge(
    left=df_store.assign(key=0),   # "key"という仮の列を作成し、デフォルト値を0とした
    right=df_product.assign(key=0),
    how="outer",
    on="key"
)

print("件数:", len(df_040))

件数: 531590


In [59]:
# 別解

df_store_tmp = df_store.copy()
df_store_tmp["key"] = 0
df_product_tmp = df_product.copy()
df_product_tmp["key"] = 0

df_040 = pd.merge(df_store_tmp, df_product_tmp, how="outer", on="key")

print("件数:", len(df_040))

件数: 531590


---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [60]:
print("集計")
df_041 = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd").sum().reset_index()
display(df_041.head())

print("前回との差(lag_ymdとlag_amount)を表示")
df_041 = pd.concat([df_041, df_041.shift(periods=1)], axis=1)
df_041.columns = ["sales_ymd", "amount", "lag_ymd", "lag_amount"]
display(df_041.head())

print("前回売上からの売上金額増減(diff_amout)の計算と10件の表示")
df_041["diff_amount"] = df_041["amount"] - df_041["lag_amount"]
df_041.head(10)

集計


,sales_ymd,amount
0,20170101,33723
1,20170102,24165
2,20170103,27503
3,20170104,36165
4,20170105,37830


前回との差(lag_ymdとlag_amount)を表示


,sales_ymd,amount,lag_ymd,lag_amount
0,20170101,33723,NaN,NaN
1,20170102,24165,20170101.0,33723.0
2,20170103,27503,20170102.0,24165.0
3,20170104,36165,20170103.0,27503.0
4,20170105,37830,20170104.0,36165.0


前回売上からの売上金額増減(diff_amout)の計算と10件の表示


,sales_ymd,amount,lag_ymd,lag_amount,diff_amount
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0
5,20170106,32387,20170105.0,37830.0,-5443.0
6,20170107,23415,20170106.0,32387.0,-8972.0
7,20170108,24737,20170107.0,23415.0,1322.0
8,20170109,26718,20170108.0,24737.0,1981.0
9,20170110,20143,20170109.0,26718.0,-6575.0


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [61]:
# filterでnull削除
shift_day = 3

original_df = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd").sum().reset_index()
df_042 = original_df.copy()

for shift in range(1, 1 + shift_day):
    tmp_df = original_df.shift(shift)
    tmp_df.columns = [f"lag_ymd{shift}", f"lag_amount{shift}"]
    df_042 = pd.concat([df_042, tmp_df], axis=1)

df_042 = df_042.dropna()

df_042.head(10)

,sales_ymd,amount,lag_ymd1,lag_amount1,lag_ymd2,lag_amount2,lag_ymd3,lag_amount3
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0
10,20170111,24287,20170110.0,20143.0,20170109.0,26718.0,20170108.0,24737.0
11,20170112,23526,20170111.0,24287.0,20170110.0,20143.0,20170109.0,26718.0
12,20170113,28004,20170112.0,23526.0,20170111.0,24287.0,20170110.0,20143.0


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [62]:
print("レシート明細データと顧客データの結合と年代（era）の追加")
df_tmp = pd.merge(
    left=df_receipt,
    right=df_customer,
    how="inner",
    on="customer_id"
)
df_tmp["era"] = df_tmp["age"].apply(lambda x: x // 10 * 10)
display(df_tmp.head(3))

print("性別コードと年代ごとに売上金額を合計した売り上げサマリを表示")

df_043 = pd.pivot_table(
    data=df_tmp,
    values="amount",      # 売上金額
    aggfunc="sum",        # 合計値
    index="era",          # 行：各年代で分ける
    columns="gender_cd",  # 列：合計値を各性別で分ける
).reset_index()

df_043.columns = ["era", "male", "female", "unknown"]
df_043

レシート明細データと顧客データの結合と年代（era）の追加


,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,20150201,E-20100908-F,20
1,20170509,1494288000,S14006,112,1,CS006214000001,P071401004,1,1100,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,20150201,E-20100908-F,20
2,20170608,1496880000,S14006,112,1,CS006214000001,P060104021,1,120,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,20150201,E-20100908-F,20


性別コードと年代ごとに売上金額を合計した売り上げサマリを表示


,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [63]:
df_043.melt(id_vars="era")

,era,variable,value
0,10,male,1591.0
1,20,male,72940.0
2,30,male,177322.0
3,40,male,19355.0
4,50,male,54320.0
5,60,male,272469.0
6,70,male,13435.0
7,80,male,46360.0
8,90,male,NaN
9,10,female,149836.0


In [64]:
df_044 = (
    df_043
    .rename(columns={"male":"00", "female":"01", "unknown":"99"})  # カラム名の変換
    .melt(
        id_vars="era",         # 行：era
        var_name="gender_cd",  # variable列の名前：gender_cd
        value_name="amount"    # value列の名前：amount
    )
    .sort_values(by=["era", "gender_cd"])
    .reset_index(drop=True)
    .dropna()
)

df_044

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


In [65]:
# 別解

df_044 = (
    df_043
    .set_index("era")
    .stack()
    .reset_index()
    .replace({"male": "00", "female": "01", "unknown": 99})
    .rename(columns={"lebel_1": "gender_cd", 0: "amount"})
)

df_044

,era,level_1,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


## Date型

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [66]:
df_045 = pd.concat([
    df_customer["customer_id"],
    pd.to_datetime(df_customer["birth_day"]).dt.strftime("%Y%m%d")  # Date型のフォーマットの変更
], axis=1)

df_045.head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [67]:
df_046 = pd.concat([
    df_customer["customer_id"],
    pd.to_datetime(df_customer["application_date"])  # String型をDate型へ変換
], axis=1)  # 列に追加

df_046.head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [68]:
df_047 = pd.concat([
    pd.to_datetime(df_receipt["sales_ymd"].astype("str")),  # Int型→Str型→Date型へ変換
    df_receipt[["receipt_no", "receipt_sub_no"]]
], axis=1)

df_047.head(10)

,sales_ymd,receipt_no,receipt_sub_no
0,2018-11-03,112,1
1,2018-11-18,1132,2
2,2017-07-12,1102,1
3,2019-02-05,1132,1
4,2018-08-21,1102,2
5,2019-06-05,1112,1
6,2018-12-05,1102,2
7,2019-09-22,1102,1
8,2017-05-04,1112,2
9,2019-10-10,1102,1


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [69]:
df_048 = pd.concat([
    pd.to_datetime(df_receipt["sales_epoch"], unit="s"),  # Int型(UNIX秒) -> Date型
    df_receipt[["receipt_no", "receipt_sub_no"]]
], axis=1)

df_048.head(10)

,sales_epoch,receipt_no,receipt_sub_no
0,2018-11-03,112,1
1,2018-11-18,1132,2
2,2017-07-12,1102,1
3,2019-02-05,1132,1
4,2018-08-21,1102,2
5,2019-06-05,1112,1
6,2018-12-05,1102,2
7,2019-09-22,1102,1
8,2017-05-04,1112,2
9,2019-10-10,1102,1


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [70]:
df_049 = pd.concat([
    # Int型 -> Date型 -> dt propertyからyear propertyの抜き出し (+名前変更)
    pd.to_datetime(df_receipt["sales_epoch"], unit="s").dt.year.rename("sales_year"),
    df_receipt[["receipt_no", "receipt_sub_no"]]
], axis=1)

df_049.head(10)

,sales_year,receipt_no,receipt_sub_no
0,2018,112,1
1,2018,1132,2
2,2017,1102,1
3,2019,1132,1
4,2018,1102,2
5,2019,1112,1
6,2018,1102,2
7,2019,1102,1
8,2017,1112,2
9,2019,1102,1


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [71]:
df_050 = pd.concat([
    # Int型 -> Date型 -> dt propertyをstrftime()メソッドでフォーマット変更 (+名前変更)
    pd.to_datetime(df_receipt["sales_epoch"], unit="s").dt.strftime("%m").rename("sales_month"),
    df_receipt[["receipt_no", "receipt_sub_no"]]
], axis=1)

df_050.head(10)

,sales_month,receipt_no,receipt_sub_no
0,11,112,1
1,11,1132,2
2,07,1102,1
3,02,1132,1
4,08,1102,2
5,06,1112,1
6,12,1102,2
7,09,1102,1
8,05,1112,2
9,10,1102,1


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [72]:
df_051 = pd.concat([
    # Int型 -> Date型 -> dt propertyをstrftime()メソッドでフォーマット変更 (+名前変更)
    pd.to_datetime(df_receipt["sales_epoch"], unit="s").dt.strftime("%d").rename("sales_day"),
    df_receipt[["receipt_no", "receipt_sub_no"]]
], axis=1)

df_051.head(10)

,sales_day,receipt_no,receipt_sub_no
0,03,112,1
1,18,1132,2
2,12,1102,1
3,05,1132,1
4,21,1102,2
5,05,1112,1
6,05,1102,2
7,22,1102,1
8,04,1112,2
9,10,1102,1


## 条件分岐
.when()や.apply()を用いる

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [73]:
df_tmp = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]  # Zから始まるものを除外
# 顧客IDごとに売上金額を合計
df_052 = df_tmp[["customer_id", "amount"]].groupby("customer_id").sum().reset_index()
# 2000以下を0, 2000以上を1に二値化
# np.where(条件, Trueの時の処理, Falseの時の処理)を利用
df_052["sales_flag"] = np.where(df_052["amount"] > 2000, 1, 0)

df_052.head(10)

,customer_id,amount,sales_flag
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


In [74]:
# 別解：apply()を利用
df_tmp = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]  # Zから始まるものを除外
# 顧客IDごとに売上金額を合計
df_052 = df_tmp[["customer_id", "amount"]].groupby("customer_id").sum().reset_index()
# 2000以下を0, 2000以上を1に二値化
df_052["sales_flag"] = df_052["amount"].apply(lambda x: 1 if x > 2000 else 0)

df_052.head(10)

,customer_id,amount,sales_flag
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [75]:
df_customer.head(2)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0


In [76]:
df_053 = df_customer[["customer_id", "postal_cd"]].copy()

df_053["postal_flag"] = df_053["postal_cd"].apply(
        lambda x: 1 if 100 < int(x[:3]) < 210 else 0
)

df_053 = pd.merge(
    # 結合して売り上げ実績のある顧客情報を取得
    left = df_receipt,
    right=df_053,
    how="inner",
    on="customer_id"
).groupby("postal_flag").aggregate({"customer_id": "nunique"})  # postal_flagでグルーピングし、nuniqueで頻出数をカウント

df_053

,customer_id
postal_flag,
0,3906
1,4400


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [77]:
# 顧客IDと住所
df_054 = df_customer[["customer_id", "address"]].copy()
# addressの3文字目までで判別(map関数を用いるｄ)
df_054["prefecture_code"] = df_054["address"].str[0:3].map(
    {"埼玉県": "11", "千葉県": "12", "東京都": "13", "神奈川": "14"}
)
df_054.head(10)

,customer_id,address,prefecture_code
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


In [78]:
# 別解（正規表現を利用）
df_054 = df_customer[["customer_id", "address"]].copy()
# addressの3文字目までで判別(map関数を用いるｄ)
df_054["prefecture_code"] = df_054["address"].str.extract(r"(^.*?[都道府県])")[0].map(
    {"埼玉県": "11", "千葉県": "12", "東京都": "13", "神奈川県": "14"}
)
df_054.head(10)

,customer_id,address,prefecture_code
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [79]:
# 売り上げの合計を顧客IDごとに取得
df_055 = df_receipt[["customer_id", "amount"]].groupby("customer_id").sum().reset_index()

pct25 = np.quantile(df_055["amount"], 0.25)
pct50 = np.quantile(df_055["amount"], 0.50)
pct75 = np.quantile(df_055["amount"], 0.75)

def identify_category(amount):
    if amount < pct25:
        return 1
    elif pct25 <= amount < pct50:
        return 2
    elif pct50 <= amount < pct75:
        return 3
    else:
        return 4

df_055["amount_category"] = df_055["amount"].apply(identify_category)

df_055.head(10)

,customer_id,amount,amount_category
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


In [80]:
# 別解（cutを利用）

df_055 = df_receipt[["customer_id", "amount"]].groupby("customer_id").sum().reset_index()

pct25 = np.quantile(df_055["amount"], 0.25)
pct50 = np.quantile(df_055["amount"], 0.50)
pct75 = np.quantile(df_055["amount"], 0.75)
pct_max = df_055["amount"].max()

df_055["quantile"] = pd.cut(
    df_055["amount"], [0, pct25, pct50, pct75, pct_max+1], right=False
)
df_055["amount_category"] = df_055.groupby("quantile", observed=True).ngroup() + 1

df_055.head(10)

,customer_id,amount,quantile,amount_category
0,CS001113000004,1298,"[548.5, 1478.0)",2
1,CS001114000005,626,"[548.5, 1478.0)",2
2,CS001115000010,3044,"[1478.0, 3651.0)",3
3,CS001205000004,1988,"[1478.0, 3651.0)",3
4,CS001205000006,3337,"[1478.0, 3651.0)",3
5,CS001211000025,456,"[0.0, 548.5)",1
6,CS001212000027,448,"[0.0, 548.5)",1
7,CS001212000031,296,"[0.0, 548.5)",1
8,CS001212000046,228,"[0.0, 548.5)",1
9,CS001212000070,456,"[0.0, 548.5)",1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [81]:
df_056 = df_customer[["customer_id", "birth_day"]].copy()

df_056["era"] = df_customer["age"].apply(lambda age: min(age // 10 * 10, 60))
df_056.head(10)

,customer_id,birth_day,era
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [82]:
df_057 = df_056.copy()
df_057["gender_era"] = df_customer["gender_cd"] + df_056["era"].astype(str)

df_057.head(10)

,customer_id,birth_day,era,gender_era
0,CS021313000114,1981-04-29,30,130
1,CS037613000071,1952-04-01,60,960
2,CS031415000172,1976-10-04,40,140
3,CS028811000001,1933-03-27,60,160
4,CS001215000145,1995-03-29,20,120
5,CS020401000016,1974-09-15,40,040
6,CS015414000103,1977-08-09,40,140
7,CS029403000008,1973-08-17,40,040
8,CS015804000004,1931-05-02,60,060
9,CS033513000180,1962-07-11,50,150


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [83]:
df_058 = pd.get_dummies(
    data=df_customer[["customer_id", "gender_cd"]],
    columns=["gender_cd"],
    dtype=int
)

df_058.head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


## 統計処理

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

- 標準化
$$
x_{new}^i = \frac{x^i - \mu}{\sigma}
$$

In [84]:
df_059 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})  # amountの合計
    .reset_index()
)

mean = df_059["amount"].mean()
std = df_059["amount"].std()

df_059["standardized_amount"] = (df_059["amount"] - mean) / std

df_059.head(10)

,customer_id,amount,standardized_amount
0,CS001113000004,1298,-0.459350
1,CS001114000005,626,-0.706348
2,CS001115000010,3044,0.182403
3,CS001205000004,1988,-0.205737
4,CS001205000006,3337,0.290096
5,CS001211000025,456,-0.768832
6,CS001212000027,448,-0.771773
7,CS001212000031,296,-0.827641
8,CS001212000046,228,-0.852635
9,CS001212000070,456,-0.768832


In [85]:
# 別解 scikit-learn使用
from sklearn.preprocessing import StandardScaler

df_059 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})  # amountの合計
    .reset_index()
)

scaler = StandardScaler()

df_059["standardized_amount"] = scaler.fit_transform(df_059[["amount"]])

df_059.head(10)

,customer_id,amount,standardized_amount
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

- 正規化
$$
x_{new}^i = \frac{x^i - x_{min}}{x_{max} - x_{min}}
$$

In [86]:
df_060 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})  # amountの合計
    .reset_index()
)

min = df_060["amount"].min()
max = df_060["amount"].max()

df_060["normalized_amount"] = (df_060["amount"] - min) / (max - min)

df_060.head(10)

,customer_id,amount,normalized_amount
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


In [87]:
# 別解
from sklearn.preprocessing import MinMaxScaler

df_060 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})  # amountの合計
    .reset_index()
)

scaler = MinMaxScaler()

df_060["normalized_amount"] = scaler.fit_transform(df_060[["amount"]])

df_060.head(10)

,customer_id,amount,normalized_amount
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [88]:
df_061 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})
    .reset_index()
)

df_061["log10_amount"] = np.log10(df_061["amount"])

df_061.head(10)

,customer_id,amount,log10_amount
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [89]:
df_062 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id").aggregate({"amount": "sum"})
    .reset_index()
)

df_062["log_amount"] = np.log(df_061["amount"])

df_062.head(10)

,customer_id,amount,log_amount
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [90]:
df_063 = df_product.copy()

df_063["unit_profit"] = df_063["unit_price"] - df_063["unit_cost"]

df_063.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit
0,P040101001,04,0401,040101,198.0,149.0,49.0
1,P040101002,04,0401,040101,218.0,164.0,54.0
2,P040101003,04,0401,040101,230.0,173.0,57.0
3,P040101004,04,0401,040101,248.0,186.0,62.0
4,P040101005,04,0401,040101,268.0,201.0,67.0
5,P040101006,04,0401,040101,298.0,224.0,74.0
6,P040101007,04,0401,040101,338.0,254.0,84.0
7,P040101008,04,0401,040101,420.0,315.0,105.0
8,P040101009,04,0401,040101,498.0,374.0,124.0
9,P040101010,04,0401,040101,580.0,435.0,145.0


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [91]:
df_064 = df_product.copy()

df_064["unit_profit_rate"] = (df_064["unit_price"] - df_064["unit_cost"]) / df_064["unit_price"]

print("利益率の全体平均")
df_064["unit_profit_rate"].mean(skipna=True)

利益率の全体平均


0.24911389885177

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [92]:
df_065 = df_product[["unit_price", "unit_cost"]].copy()
df_065["new_price"] = np.floor(df_065["unit_cost"] / 0.7)

df_065["new_unit_profit_rate"] = (df_065["new_price"] - df_065["unit_cost"]) / df_065["new_price"]

df_065

,unit_price,unit_cost,new_price,new_unit_profit_rate
0,198.0,149.0,212.0,0.297170
1,218.0,164.0,234.0,0.299145
2,230.0,173.0,247.0,0.299595
3,248.0,186.0,265.0,0.298113
4,268.0,201.0,287.0,0.299652
...,...,...,...,...
10025,280.0,210.0,300.0,0.300000
10026,680.0,510.0,728.0,0.299451
10027,1080.0,810.0,1157.0,0.299914
10028,1130.0,848.0,1211.0,0.299752


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [93]:
df_066 = df_product[["unit_price", "unit_cost"]].copy()
df_066["new_price"] = np.round(df_066["unit_cost"] / 0.7)

df_066["new_unit_profit_rate"] = (df_066["new_price"] - df_066["unit_cost"]) / df_066["new_price"]

df_065

,unit_price,unit_cost,new_price,new_unit_profit_rate
0,198.0,149.0,212.0,0.297170
1,218.0,164.0,234.0,0.299145
2,230.0,173.0,247.0,0.299595
3,248.0,186.0,265.0,0.298113
4,268.0,201.0,287.0,0.299652
...,...,...,...,...
10025,280.0,210.0,300.0,0.300000
10026,680.0,510.0,728.0,0.299451
10027,1080.0,810.0,1157.0,0.299914
10028,1130.0,848.0,1211.0,0.299752


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [94]:
df_067 = df_product[["unit_price", "unit_cost"]].copy()
df_067["new_price"] = np.ceil(df_067["unit_cost"] / 0.7)

df_067["new_unit_profit_rate"] = (df_067["new_price"] - df_067["unit_cost"]) / df_067["new_price"]

df_067.head(10)

,unit_price,unit_cost,new_price,new_unit_profit_rate
0,198.0,149.0,213.0,0.300469
1,218.0,164.0,235.0,0.302128
2,230.0,173.0,248.0,0.302419
3,248.0,186.0,266.0,0.300752
4,268.0,201.0,288.0,0.302083
5,298.0,224.0,320.0,0.300000
6,338.0,254.0,363.0,0.300275
7,420.0,315.0,451.0,0.301552
8,498.0,374.0,535.0,0.300935
9,580.0,435.0,622.0,0.300643


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [95]:
df_068 = df_product[["unit_price", "unit_cost"]].copy()
df_068["price_with_tax"] = np.floor(df_068["unit_cost"] * 1.1)

df_068.head(10)

,unit_price,unit_cost,price_with_tax
0,198.0,149.0,163.0
1,218.0,164.0,180.0
2,230.0,173.0,190.0
3,248.0,186.0,204.0
4,268.0,201.0,221.0
5,298.0,224.0,246.0
6,338.0,254.0,279.0
7,420.0,315.0,346.0
8,498.0,374.0,411.0
9,580.0,435.0,478.0


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [96]:
# 顧客データの合計
df_tmp1 = (
    df_receipt
    .groupby("customer_id").aggregate({"amount": "sum"})
    .rename(columns={"amount": "sum_amount"})
    .reset_index()
)
# 顧客データの中でカテゴリ大区分コードが"07"の合計値
df_tmp2 = (
    pd.merge(
        left=df_receipt,
        right=df_product[df_product["category_major_cd"] == "07"],
        how="inner", on="product_cd"
    )
    .groupby("customer_id").aggregate({"amount": "sum"})
    .rename(columns={"amount": "sum_amount_07"})
    .reset_index()
)

df_069 = pd.merge(df_tmp1, df_tmp2, how="inner", on="customer_id")
df_069["sales_07_rate"] = df_069["sum_amount_07"] / df_069["sum_amount"]

df_069.head(10)

,customer_id,sum_amount,sum_amount_07,sales_07_rate
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


## Date型の計算

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [97]:
df_070 = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
# 売上日と入会申し込み日
df_070 = pd.merge(left=df_070, right=df_customer[["customer_id", "application_date"]],
                  how="inner", on="customer_id")

# int型 -> str型 -> date型
df_070["sales_ymd"] = pd.to_datetime(df_070["sales_ymd"].astype(str))

# str型-> date型
df_070["application_date"] = pd.to_datetime(df_070["application_date"])

# 日付の差 -> dt.daysでdayだけ取り出し
df_070["elapsed_days"] = (df_070["sales_ymd"] - df_070["application_date"]).dt.days

df_070.head(10)

,customer_id,sales_ymd,application_date,elapsed_days
0,CS006214000001,2018-11-03,2015-02-01,1371
1,CS006214000001,2017-05-09,2015-02-01,828
2,CS006214000001,2017-06-08,2015-02-01,858
3,CS006214000001,2018-10-28,2015-02-01,1365
4,CS006214000001,2019-09-08,2015-02-01,1680
5,CS006214000001,2018-01-31,2015-02-01,1095
6,CS006214000001,2017-07-05,2015-02-01,885
7,CS006214000001,2018-11-10,2015-02-01,1378
8,CS006214000001,2019-04-10,2015-02-01,1529
9,CS006214000001,2019-06-01,2015-02-01,1581


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [98]:
df_071 = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
# 売上日と入会申し込み日
df_071 = pd.merge(left=df_071, right=df_customer[["customer_id", "application_date"]],
                  how="inner", on="customer_id")

# int型 -> str型 -> date型
df_071["sales_ymd"] = pd.to_datetime(df_071["sales_ymd"].astype(str))

# str型-> date型
df_071["application_date"] = pd.to_datetime(df_071["application_date"])

df_071["elapsed_months"] = (
    (df_071["sales_ymd"].dt.year - df_071["application_date"].dt.year)*12  # year -> month
    + (df_071["sales_ymd"].dt.month - df_071["application_date"].dt.month)
)

df_071.head(10)

,customer_id,sales_ymd,application_date,elapsed_months
0,CS006214000001,2018-11-03,2015-02-01,45
1,CS006214000001,2017-05-09,2015-02-01,27
2,CS006214000001,2017-06-08,2015-02-01,28
3,CS006214000001,2018-10-28,2015-02-01,44
4,CS006214000001,2019-09-08,2015-02-01,55
5,CS006214000001,2018-01-31,2015-02-01,35
6,CS006214000001,2017-07-05,2015-02-01,29
7,CS006214000001,2018-11-10,2015-02-01,45
8,CS006214000001,2019-04-10,2015-02-01,50
9,CS006214000001,2019-06-01,2015-02-01,52


---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [99]:
df_072 = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
# 売上日と入会申し込み日
df_072 = pd.merge(left=df_072, right=df_customer[["customer_id", "application_date"]],
                  how="inner", on="customer_id")

# int型 -> str型 -> date型
df_072["sales_ymd"] = pd.to_datetime(df_072["sales_ymd"].astype(str))

# str型-> date型
df_072["application_date"] = pd.to_datetime(df_072["application_date"])

df_072["elapsed_years"] = (
    df_072["sales_ymd"].dt.year - df_072["application_date"].dt.year
)

df_072.head(10)

,customer_id,sales_ymd,application_date,elapsed_years
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
3,CS006214000001,2018-10-28,2015-02-01,3
4,CS006214000001,2019-09-08,2015-02-01,4
5,CS006214000001,2018-01-31,2015-02-01,3
6,CS006214000001,2017-07-05,2015-02-01,2
7,CS006214000001,2018-11-10,2015-02-01,3
8,CS006214000001,2019-04-10,2015-02-01,4
9,CS006214000001,2019-06-01,2015-02-01,4


---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [100]:
df_073 = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
# 売上日と入会申し込み日
df_073 = pd.merge(left=df_073, right=df_customer[["customer_id", "application_date"]],
                  how="inner", on="customer_id")

# int型 -> str型 -> date型
df_073["sales_ymd"] = pd.to_datetime(df_073["sales_ymd"].astype(str))

# str型-> date型
df_073["application_date"] = pd.to_datetime(df_073["application_date"])

df_073["elapsed_epoch"] = (
    (
        df_073["sales_ymd"].view(np.int64) / 10**9
        - df_073["application_date"].view(np.int64) / 10**9
    )
)

df_073.head(10)

,customer_id,sales_ymd,application_date,elapsed_epoch
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
3,CS006214000001,2018-10-28,2015-02-01,117936000.0
4,CS006214000001,2019-09-08,2015-02-01,145152000.0
5,CS006214000001,2018-01-31,2015-02-01,94608000.0
6,CS006214000001,2017-07-05,2015-02-01,76464000.0
7,CS006214000001,2018-11-10,2015-02-01,119059200.0
8,CS006214000001,2019-04-10,2015-02-01,132105600.0
9,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [101]:
import calendar
import datetime

df_074 = df_receipt[["sales_ymd"]].copy()

df_074["sales_ymd"] = pd.to_datetime(df_074["sales_ymd"].astype(str))

df_074["week"] = df_074["sales_ymd"].apply(lambda day: calendar.day_name[day.weekday()])
# Monday -> 0, Tuesday -> 1, ... , Sunday -> 6
df_074["elapsed_days"] = df_074["sales_ymd"].apply(lambda day: day.weekday())

df_074["monday"] = df_074["sales_ymd"].apply(lambda day: day - datetime.timedelta(days=day.weekday()))

df_074

,sales_ymd,week,elapsed_days,monday
0,2018-11-03,Saturday,5,2018-10-29
1,2018-11-18,Sunday,6,2018-11-12
2,2017-07-12,Wednesday,2,2017-07-10
3,2019-02-05,Tuesday,1,2019-02-04
4,2018-08-21,Tuesday,1,2018-08-20
...,...,...,...,...
104676,2018-02-21,Wednesday,2,2018-02-19
104677,2019-09-11,Wednesday,2,2019-09-09
104678,2017-03-11,Saturday,5,2017-03-06
104679,2017-03-31,Friday,4,2017-03-27


## ランダム抽出

---
> P-075:顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [102]:
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2193,CS019315000082,秦 早紀,1,女性,1984-06-09,34,176-0005,東京都練馬区旭丘**********,S13019,20150831,1-20080514-3
504,CS002715000073,奥野 小百合,1,女性,1944-08-29,74,187-0021,東京都小平市上水南町**********,S13002,20150423,0-00000000-0
10541,CS001413000192,永野 みあ,1,女性,1971-02-27,48,144-0056,東京都大田区西六郷**********,S13001,20150711,1-20080317-1
6561,CS008515000124,原 あや子,1,女性,1963-04-04,55,157-0073,東京都世田谷区砧**********,S13008,20150916,A-20100807-C
21218,CS009412000134,菅原 瞬,9,不明,1976-11-27,42,156-0052,東京都世田谷区経堂**********,S13009,20150410,0-00000000-0
19234,CS016612000043,田崎 真悠子,1,女性,1950-07-10,68,184-0014,東京都小金井市貫井南町**********,S13016,20150711,0-00000000-0
8136,CS032315000034,瀬川 沙知絵,1,女性,1986-07-09,32,144-0055,東京都大田区仲六郷**********,S13032,20150607,0-00000000-0
12188,CS004514000020,野沢 さとみ,9,不明,1966-02-09,53,165-0033,東京都中野区若宮**********,S13004,20141204,8-20090817-1
9582,CS012415000144,内村 真奈美,1,女性,1975-05-16,43,231-0838,神奈川県横浜市中区豆口台**********,S14012,20150713,E-20100528-E
16710,CS002415000905,薬師丸 早紀,1,女性,1972-11-22,46,185-0014,東京都国分寺市東恋ケ窪**********,S13002,20170424,0-00000000-0


---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [103]:
_, df_076 = train_test_split(df_customer, test_size=0.1,
                             stratify=df_customer['gender_cd'])

df_076.groupby('gender_cd').agg({'customer_id' : 'count'})

,customer_id
gender_cd,
0,298
1,1793
9,107


## 外れ値と欠損値

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [104]:
# レシート明細データの売り上げ金額を顧客単位に合計
df_077 = df_receipt.groupby("customer_id").aggregate({"amount": "sum"}).reset_index()

# 売上金額合計を対数化（常用対数）
df_077["log_sum_amount"] = np.log10(df_077["amount"])

mean = df_077["log_sum_amount"].mean()
std = df_077["log_sum_amount"].std()

# 外れ値（平均から3σを超えたもの）の抽出
df_077[(df_077["log_sum_amount"] - mean).abs() > 3 * std]

,customer_id,amount,log_sum_amount
8306,ZZ000000000000,12395003,7.093247


---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [105]:
df_078 = (
    df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
    .groupby("customer_id")
    .aggregate({"amount": "sum"})
    .reset_index()
)

pct25 = np.quantile(df_078["amount"], q=0.25)
pct75 = np.quantile(df_078["amount"], q=0.75)
iqr = pct75 - pct25

# 外れ値の抽出
df_078 = df_078[
    (pct25 - 1.5 * iqr > df_078["amount"])
    | (df_078["amount"] > pct75 + 1.5 * iqr)
]

df_078.head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [106]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [107]:
df_080 = df_product.dropna()

print("削除前:", len(df_product))
print("削除後:", len(df_080))

削除前: 10030
削除後: 10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [108]:
df_081 = df_product.fillna(
    {
        "unit_price": np.round(np.nanmean(df_product["unit_price"])),
        "unit_cost": np.round(np.nanmean(df_product["unit_cost"]))
    }
)
# 欠損が生じていないことの確認
df_081.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [109]:
df_082 = df_product.fillna(
    {
        "unit_price": np.round(np.nanmedian(df_product["unit_price"])),
        "unit_cost": np.round(np.nanmedian(df_product["unit_cost"]))
    }
)
# 欠損が生じていないことの確認
df_082.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [137]:
df_083 = (
    df_product.groupby("category_small_cd")
    # 中央値を新しい列名で保存
    .aggregate(
        median_price=("unit_price", "median"),
        median_cost=("unit_cost", "median")
    )
    .reset_index()
)
# 商品データと結合（median_price, median_cost列が追加した商品データ）
df_083 = pd.merge(
    left=df_product, right=df_083, how="inner", on="category_small_cd"
)

# 欠損値(nan)をmedian値で置換
df_083["unit_price"] = (
    df_083[["unit_price", "median_price"]]
    .apply(lambda price: np.round(price.iloc[1]) if np.isnan(price.iloc[0]) else price.iloc[0], axis=1)
)

df_083["unit_cost"] = (
    df_083[["unit_cost", "median_cost"]]
    .apply(lambda cost: np.round(cost.iloc[1]) if np.isnan(cost.iloc[0]) else cost.iloc[0], axis=1)
)

df_083.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64

In [138]:
# 別解(mask)利用

df_083 = (
    df_product.groupby("category_small_cd")
    # 中央値を新しい列名で保存
    .aggregate(
        median_price=("unit_price", "median"),
        median_cost=("unit_cost", "median")
    )
    .reset_index()
)
# 商品データと結合（median_price, median_cost列が追加した商品データ）
df_083 = pd.merge(
    left=df_product, right=df_083, how="inner", on="category_small_cd"
)

# 欠損値(nan)をmedian値で置換
df_083["unit_price"] = (
    df_083["unit_price"].mask(df_083["unit_price"].isnull(), df_083["median_price"])
)

df_083["unit_cost"] = (
    df_083["unit_cost"].mask(df_083["unit_cost"].isnull(), df_083["median_cost"])
)

df_083.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64

In [139]:
# 別解2 (fillna, transform)

df_083 = df_product.copy()

for col in ["unit_price", "unit_cost"]:
    df_083[col] = (
        df_083[col].fillna(df_083.groupby("category_small_cd")[col].transform("median").round())
    )

df_083.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

## 応用

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [150]:
# 2019年の売り上げ
df_2019 = (
    df_receipt[(20190101 <= df_receipt["sales_ymd"]) & (df_receipt["sales_ymd"] <= 20191231)]
    .groupby("customer_id")
    .aggregate(amount_2019=("amount", "sum"))
    .reset_index()
)

# 全期間の売り上げ
df_all = df_receipt.groupby("customer_id").aggregate(amount_all=("amount", "sum")).reset_index()

# 結合
df_084 = (
    df_customer[["customer_id"]]
    .merge(right=df_2019, how="left", on="customer_id")
    .merge(right=df_all, how="left", on="customer_id")
)

# 0埋め
df_084["amount_2019"] = df_084["amount_2019"].fillna(0)
df_084["amount_all"] = df_084["amount_all"].fillna(0)

# 比率(2019年のものが0なら0を記述)
df_084["amount_rate"] = (
    df_084[["amount_2019", "amount_all"]]
    .apply(lambda amount: 0 if amount.iloc[0] == 0 else amount.iloc[0]/amount.iloc[1], axis=1)
)
print("nanの数")
print(df_084.isnull().sum())

df_084[df_084["amount_rate"] > 0].head(10)

nanの数
customer_id    0
amount_2019    0
amount_all     0
amount_rate    0
dtype: int64


,customer_id,amount_2019,amount_all,amount_rate
2,CS031415000172,2971.0,5088.0,0.583923
6,CS015414000103,874.0,3122.0,0.279949
12,CS011215000048,248.0,3444.0,0.072009
15,CS029415000023,3767.0,5167.0,0.729050
21,CS035415000029,5823.0,7504.0,0.775986
23,CS023513000066,208.0,771.0,0.269780
24,CS035513000134,463.0,1565.0,0.295847
27,CS001515000263,216.0,216.0,1.000000
30,CS006415000279,229.0,229.0,1.000000
32,CS031415000106,215.0,7741.0,0.027774


---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [152]:
df_085 = (
    df_geocode.groupby("postal_cd")
    .aggregate(mean_longitude=("longitude", "mean"), mean_latitude=("latitude", "mean"))
    .reset_index()
)

df_085 = df_085.merge(df_customer, how="inner", on="postal_cd")

df_085.head(10)

,postal_cd,mean_longitude,mean_latitude,customer_id,customer_name,gender_cd,gender,birth_day,age,address,application_store_cd,application_date,status_cd
0,101-0025,139.77983,35.69766,CS037211000016,大森 杏,1,女性,1993-04-24,25,東京都千代田区神田佐久間町**********,S13037,20150531,0-00000000-0
1,101-0025,139.77983,35.69766,CS015301000001,米倉 聖陽,0,男性,1982-03-23,37,東京都千代田区神田佐久間町**********,S13015,20150216,0-00000000-0
2,101-0031,139.78015,35.69619,CS037112000007,寺西 さやか,1,女性,2004-01-20,15,東京都千代田区東神田**********,S13037,20151108,0-00000000-0
3,101-0031,139.78015,35.69619,CS015311000003,藤沢 知世,1,女性,1985-08-12,33,東京都千代田区東神田**********,S13015,20141226,0-00000000-0
4,101-0032,139.77763,35.69234,CS015301000007,大畑 秀隆,0,男性,1983-08-09,35,東京都千代田区岩本町**********,S13015,20151014,0-00000000-0
5,101-0038,139.77451,35.69085,CS015211000006,上杉 佳乃,1,女性,1994-02-25,25,東京都千代田区神田美倉町**********,S13015,20150617,A-20100102-A
6,101-0041,139.76984,35.69622,CS037211000010,堤 あさみ,1,女性,1997-05-31,21,東京都千代田区神田須田町**********,S13037,20150628,9-20100707-9
7,102-0074,139.74570,35.69314,CS031311000016,北田 佳乃,1,女性,1979-06-25,39,東京都千代田区九段南**********,S13031,20150626,0-00000000-0
8,102-0083,139.73669,35.68364,CS031301000012,秋葉 進,0,男性,1982-07-10,36,東京都千代田区麹町**********,S13031,20150702,0-00000000-0
9,102-0083,139.73669,35.68364,CS031211000016,河内 由宇,1,女性,1990-06-02,28,東京都千代田区麹町**********,S13031,20150413,0-00000000-0


---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\begin{aligned}
& longitude(radian)：\phi \\
& latitude(radian)：\lambda \\
& distance, L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2 + \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
\end{aligned}
$$

In [155]:
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(x1))
                    * math.sin(math.radians(x2))
                    + math.cos(math.radians(x1))
                    * math.cos(math.radians(x2))
                    * math.cos(math.radians(y1) - math.radians(y2)))
    return distance

df_086 = (
    pd.merge(df_085, df_store, how="inner", left_on="application_store_cd", right_on="store_cd")
    .rename(columns={"address_x": "customer_address", "address_y": "store_address"})
)

df_086["distance"] = (
    df_086[["mean_latitude", "mean_longitude", "latitude", "longitude"]]
    .apply(lambda x: calc_distance(x.iloc[0], x.iloc[1], x.iloc[2], x.iloc[3]), axis=1)
)

df_086[["customer_id", "customer_address", "store_address", "distance"]].head(10)

,customer_id,customer_address,store_address,distance
0,CS037211000016,東京都千代田区神田佐久間町**********,東京都江東区南砂一丁目,4.298502
1,CS037112000007,東京都千代田区東神田**********,東京都江東区南砂一丁目,4.196232
2,CS037211000010,東京都千代田区神田須田町**********,東京都江東区南砂一丁目,5.044078
3,CS037202000002,東京都中央区新川**********,東京都江東区南砂一丁目,3.336625
4,CS037512000101,東京都江東区毛利**********,東京都江東区南砂一丁目,1.368705
5,CS037515000175,東京都江東区毛利**********,東京都江東区南砂一丁目,1.368705
6,CS037502000046,東京都江東区毛利**********,東京都江東区南砂一丁目,1.368705
7,CS037413000287,東京都江東区住吉**********,東京都江東区南砂一丁目,1.347868
8,CS037613000069,東京都江東区住吉**********,東京都江東区南砂一丁目,1.347868
9,CS037615000020,東京都江東区住吉**********,東京都江東区南砂一丁目,1.347868


---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [157]:
df_087 = (
    df_receipt
    .groupby("customer_id").aggregate(sum_amount=("amount", "sum"))
    .reset_index()
)

df_087 = df_087.merge(df_customer, how="right", on="customer_id")

df_087["sum_amount"] = df_087["sum_amount"].fillna(0)
df_087 = df_087.sort_values(["sum_amount", "customer_id"], ascending=[False, True])

df_087.drop_duplicates(subset=["customer_name", "postal_cd"], keep="first", inplace=True)

print(
    "df_cunstomer_cnt:", len(df_customer),
    "df_087_cnt:", len(df_087),
    "diff:", len(df_customer) - len(df_087)
)


df_cunstomer_cnt: 21971 df_087_cnt: 21941 diff: 30


---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [161]:
df_088 = (
    pd.merge(
        df_customer, df_087[["customer_name", "postal_cd", "customer_id"]],
        how="inner", on=["customer_name", "postal_cd"]
    )
)

df_088.rename(
    columns={"customer_id_x": "customer_id", "customer_id_y": "integration_id"},
    inplace=True
)

print("ID数の差:", len(df_088["customer_id"].unique()) - len(df_088["integration_id"].unique()))

ID数の差: 30


---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [163]:
df_089 = (
    df_receipt
    .groupby("customer_id").aggregate({"amount": "sum"})
    .reset_index()
)

df_089 = df_089[df_089["amount"] > 0]

df_089 = pd.merge(df_customer, df_089["customer_id"], how="inner", on="customer_id")

df_train, df_test = train_test_split(
    df_089, test_size=0.2, random_state=71
)

print(f"学習データ割合: {len(df_train)/len(df_089)}")
print(f"テストデータ割合: {len(df_test)/len(df_089)}")

学習データ割合: 0.7999036840837949
テストデータ割合: 0.20009631591620516


---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [168]:
df_090 = df_receipt[["sales_ymd", "amount"]].copy()
df_090["sales_ym"] = df_090["sales_ymd"].astype(str).str[0:6]
df_090 = df_090.groupby("sales_ym").aggregate({"amount": "sum"}).reset_index()

def split_data(df, train_size: int, test_size:int, slide_window: int, start_point:int):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start:test_start], df[test_start:test_start + test_size]

df_090_train1, df_090_test1 = split_data(df_090, train_size=12,
                                         test_size=6, slide_window=6, start_point=0)

df_090_train2, df_090_test2 = split_data(df_090, train_size=12,
                                         test_size=6, slide_window=6, start_point=1)

df_090_train3, df_090_test3 = split_data(df_090, train_size=12,
                                         test_size=6, slide_window=6, start_point=2)

display(df_090_train1)


,sales_ym,amount
0,201701,902056
1,201702,764413
2,201703,962945
3,201704,847566
4,201705,884010
5,201706,894242
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739


In [167]:
display(df_090_test1)

,sales_ym,amount
12,201801,944509
13,201802,864128
14,201803,946588
15,201804,937099
16,201805,1004438
17,201806,1012329


In [169]:
df_090_train2

,sales_ym,amount
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739
10,201711,932157
11,201712,939654
12,201801,944509
13,201802,864128
14,201803,946588
15,201804,937099


In [172]:
# scikit_learnを使った方法
df_090 = df_receipt[["sales_ymd", "amount"]].copy()
df_090["sales_ym"] = df_090["sales_ymd"].astype(str).str[0:6]
df_090 = df_090.groupby("sales_ym").aggregate({"amount": "sum"}).reset_index()

tscv = TimeSeriesSplit(gap=0, max_train_size=12, n_splits=3, test_size=6)

series_list = []
for train_index, test_index in tscv.split(df_090):
    print(train_index)
    series_list.append((df_090.iloc[train_index],
                        df_090.iloc[test_index]))

df_090_train1, df_090_test1 = series_list[0]
df_090_train2, df_090_test2 = series_list[1]
df_090_train3, df_090_test3 = series_list[2]

[ 4  5  6  7  8  9 10 11 12 13 14 15]
[10 11 12 13 14 15 16 17 18 19 20 21]
[16 17 18 19 20 21 22 23 24 25 26 27]


---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [178]:
from imblearn.under_sampling import RandomUnderSampler

rs = RandomUnderSampler(random_state=71)

df_091 = (
    df_receipt
    .groupby("customer_id")
    .aggregate({"amount": "sum"})
    .reset_index()
)

df_091 = pd.merge(df_customer, df_091, how="left", on="customer_id")

df_091["is_buy_flag"] = np.where(df_091["amount"].isnull(), 0, 1)
df_091, _ = rs.fit_resample(df_091, df_091["is_buy_flag"])

print('0の件数', len(df_091[df_091["is_buy_flag"] == 0]))
print('1の件数', len(df_091[df_091["is_buy_flag"] == 1]))

0の件数 8306
1の件数 8306


---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

- 第1正規形：1つのセルには1つの値しか含まれない　list型などで登録しない
- 第2正規形：部分関数従属を排除し、完全関数従属にする　Nullとか無くす
- 第3正規形：第2正規形のテーブルから、推移的関数従属している列が切り出されたもの　段階的な従属を無くす

In [181]:
df_092_gender = df_customer[["gender_cd", "gender"]].drop_duplicates()  # genderはgender_cdの従属関係 -> 切り離す
df_092_customer = df_customer.drop(columns="gender")

display(df_092_gender)

display(df_092_customer.head(3))

,gender_cd,gender
0,1,女性
1,9,不明
5,0,男性


,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C


---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [184]:
df_093 = pd.merge(
    df_product,
    df_category[["category_small_cd", "category_major_name", "category_medium_name", "category_small_name"]],
    how="inner", on="category_small_cd"
)

df_093.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類


## ファイル入出力

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [187]:
df_093.to_csv("./../data/P_df_093_UTF-9_header.csv", encoding="utf-8", index=False)

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [188]:
df_093.to_csv("./../data/P_df_093_CP932_header.csv", encoding="cp932", index=False)

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [189]:
df_093.to_csv("./../data/P_df_093_UTF-9_noh.csv", header=False, encoding="utf-8", index=False)

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [192]:
df_097 = pd.read_csv("./../data/P_df_093_UTF-9_header.csv",
                     dtype={"category_major_cd": str,
                            "category_medium_cd": str,
                            "category_small_cd": str},
                     encoding="utf-8")

df_097.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類


---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [194]:
df_098 = pd.read_csv("./../data/P_df_093_UTF-9_noh.csv", header=None, encoding="utf-8", dtype={1: str, 2: str, 3: str})
    # columnsなしだと、1, 2,...のようなカラム名になる
df_098.columns=[
    "product_cd",
    "category_major_cd",
    "category_medium_cd",
    "category_small_cd",
    "unit_price",
    "unit_cost",
    "category_major_name",
    "category_medium_name",
    "category_small_name"
]

df_098.head(3)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類


---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [195]:
# sep="/t"でタブで分割
df_093.to_csv("./../data/P_df_093_UTF-9_header.tsv", sep="\t")

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [196]:
df_100 = pd.read_csv("./../data/P_df_093_UTF-9_header.tsv", sep="\t")

df_100.head(3)

,Unnamed: 0,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類


問題はここで終了です。お疲れ様でした。